In [15]:
import numpy as np
import gzip
import json
from scipy.stats import binned_statistic_2d
import time
import pickle
import networkx as nx

import torch
import torch.nn.functional as F
from numpy.linalg import eigvals
from torch_geometric.utils import (get_laplacian, to_scipy_sparse_matrix, to_undirected, to_dense_adj)
from torch_geometric.utils.num_nodes import maybe_num_nodes
# from torch_scatter import scatter_add

# Scipy eigendecomposition
from scipy.sparse.linalg import eigsh

In [7]:
raw_data_dir = '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/'
data_dir = '../../data/chips/clean_data/'
design = 'xbar'
N = 13

sample_names = []
corresponding_design = []
corresponding_variant = []
for idx in range(N):
    sample_name = raw_data_dir + design + '/' + str(idx + 1) + '/'
    sample_names.append(sample_name)
    corresponding_design.append(design)
    corresponding_variant.append(idx + 1)

In [17]:
k = 10
for idx in range(N):
    print(idx)
    file_name = data_dir + str(idx) + '.bipartite.pkl'
    f = open(file_name, 'rb')
    dictionary = pickle.load(f)
    f.close()
    
    instance_idx = dictionary['instance_idx']
    net_idx = dictionary['net_idx']

    file_name = data_dir + str(idx) + '.node_features.pkl'
    f = open(file_name, 'rb')
    dictionary = pickle.load(f)
    f.close()

    num_instances = dictionary['num_instances']
    num_nets = dictionary['num_nets']

    # Construct the edge list
    net_idx += num_instances
    v1 = torch.unsqueeze(torch.Tensor(np.concatenate([instance_idx, net_idx], axis = 0)).long(), dim = 1)
    v2 = torch.unsqueeze(torch.Tensor(np.concatenate([net_idx, instance_idx], axis = 0)).long(), dim = 1)

    # v1 = torch.Tensor(instance_idx).long().view(-1, 1)
    # v2 = torch.Tensor(instance_idx).long().view(-1, 1)
    # print(v1.shape)
    # print(v2.shape)

    # undir_edge_index = torch.transpose(torch.cat([v1, v2], dim = 1), 0, 1)
    undir_edge_index = torch.transpose(torch.cat([v1, v2], dim = 1), 0, 1)
    # print(undir_edge_index.shape)

    # Create symmetric graph Laplacian
    L = to_scipy_sparse_matrix(
        *get_laplacian(undir_edge_index, normalization = "sym", num_nodes = num_instances + num_nets)
    )

    # L = to_scipy_sparse_matrix(
    #     *get_laplacian(undir_edge_index, normalization="sym", num_nodes=num_instances)
    # )

    # Sparse Scipy - Eigendecomposition
    t = time.time()
    evals, evects = eigsh(L, k = k)
    print('Computation time:', time.time() - t)

    dictionary = {
        'evals': evals,
        'evects': evects
    }
    fn = data_dir + str(idx) + '.eigen.' + str(k) + '.pkl'
    f = open(fn, "wb")
    pickle.dump(dictionary, f)
    f.close()
    print('Save file', fn)

0
Computation time: 0.31820058822631836
Save file ../../data/chips/clean_data/0.eigen.10.pkl
1
Computation time: 0.3702056407928467
Save file ../../data/chips/clean_data/1.eigen.10.pkl
2
Computation time: 0.22020316123962402
Save file ../../data/chips/clean_data/2.eigen.10.pkl
3
Computation time: 0.31203365325927734
Save file ../../data/chips/clean_data/3.eigen.10.pkl
4
Computation time: 0.2996714115142822
Save file ../../data/chips/clean_data/4.eigen.10.pkl
5
Computation time: 0.3467695713043213
Save file ../../data/chips/clean_data/5.eigen.10.pkl
6
Computation time: 0.44303131103515625
Save file ../../data/chips/clean_data/6.eigen.10.pkl
7
Computation time: 0.28621959686279297
Save file ../../data/chips/clean_data/7.eigen.10.pkl
8
Computation time: 0.30371975898742676
Save file ../../data/chips/clean_data/8.eigen.10.pkl
9
Computation time: 0.38762974739074707
Save file ../../data/chips/clean_data/9.eigen.10.pkl
10
Computation time: 0.44022297859191895
Save file ../../data/chips/clean

In [16]:
for sample in range(13):
    graph_rep = 'node_features'
    fn = data_dir + '/' + str(sample) + f'.{graph_rep}.pkl'
    f = open(fn, "rb")
    d = pickle.load(f)
    f.close()
    print('Read file', fn)

    # Connection data 
    graph_rep = 'bipartite'
    fn = data_dir + '/' + str(sample) + f'.{graph_rep}.pkl'
    f = open(fn, "rb")
    dictionary = pickle.load(f)
    f.close()
    print('Read file', fn)
    
    cell = dictionary['instance_idx']
    net = dictionary['net_idx']
    num_instances = d['num_instances']
    num_nets = d['num_nets']
    edge_index = np.array([cell, net + num_instances])
    
    G = nx.Graph(edge_index.T.tolist())
    
    cell_degrees = []
    net_degrees = []
    for node in range(num_instances):
        try:
            cell_degrees.append(G.degree[node])
        except:
            cell_degrees.append(0)

    for node in range(num_instances, num_nets + num_instances):
        try:
            net_degrees.append(G.degree[node])
        except:
            net_degrees.append(0)     
    d = {
        'cell_degrees': cell_degrees,
        'net_degrees': net_degrees
    }

    fn = data_dir + '/' + str(sample) + f'.degree.pkl'
    print(fn)
    f = open(fn, "wb")
    pickle.dump(d, f)
    f.close()

Read file ../../data/chips/clean_data//0.node_features.pkl
Read file ../../data/chips/clean_data//0.bipartite.pkl
../../data/chips/clean_data//0.degree.pkl
Read file ../../data/chips/clean_data//1.node_features.pkl
Read file ../../data/chips/clean_data//1.bipartite.pkl
../../data/chips/clean_data//1.degree.pkl
Read file ../../data/chips/clean_data//2.node_features.pkl
Read file ../../data/chips/clean_data//2.bipartite.pkl
../../data/chips/clean_data//2.degree.pkl
Read file ../../data/chips/clean_data//3.node_features.pkl
Read file ../../data/chips/clean_data//3.bipartite.pkl
../../data/chips/clean_data//3.degree.pkl
Read file ../../data/chips/clean_data//4.node_features.pkl
Read file ../../data/chips/clean_data//4.bipartite.pkl
../../data/chips/clean_data//4.degree.pkl
Read file ../../data/chips/clean_data//5.node_features.pkl
Read file ../../data/chips/clean_data//5.bipartite.pkl
../../data/chips/clean_data//5.degree.pkl
Read file ../../data/chips/clean_data//6.node_features.pkl
Read 